In [1]:
#imports
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import time
import warnings

from scipy import stats
from scipy.stats import norm

from sklearn.model_selection import train_test_split, cross_val_score, GridSearchCV
from sklearn.metrics import mean_squared_error
from sklearn import linear_model, ensemble
from sklearn.linear_model import LinearRegression, Ridge, RidgeCV, Lasso, LassoCV
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from sklearn.svm import SVR

from xgboost import XGBRegressor

warnings.filterwarnings('ignore')
%matplotlib inline

In [2]:
#set seaborn plot display options
sns.set(style='whitegrid', palette='muted', font_scale=1.3)

#color variables incase they're needed
color_b = sns.color_palette('muted').as_hex()[0]
color_g = sns.color_palette('muted').as_hex()[1]
color_r = sns.color_palette('muted').as_hex()[2]
color_p = sns.color_palette('muted').as_hex()[3]
color_y = sns.color_palette('muted').as_hex()[4]
color_c = sns.color_palette('muted').as_hex()[5]

In [3]:
train_raw = pd.read_csv('train.csv')

In [4]:
#set df to manipulate, leave raws accessible
train = train_raw.copy()

#lowercase column names
train.columns = [x.lower() for x in train.columns]

#strip whitespace
to_strip = train.select_dtypes(include='object')
train[to_strip.columns] = to_strip.apply(lambda x: x.str.strip())

In [5]:
train = train[train.grlivarea < 4000]
print(train.grlivarea.max())

3627


In [6]:
#see whats missing
missing_data = train.isnull().sum().sort_values(ascending=False)
missing_data = missing_data.reset_index()
missing_data.columns = ['variable','rows_missing']
missing_data = missing_data[missing_data['rows_missing'] > 0]
missing_data

,variable,rows_missing
0,poolqc,1451
1,miscfeature,1402
2,alley,1365
3,fence,1176
4,fireplacequ,690
5,lotfrontage,259
6,garagecond,81
7,garagetype,81
8,garageyrblt,81
9,garagefinish,81


In [7]:
#handle missing values, easy part
train['poolqc'].fillna('none', inplace=True)
train['miscfeature'].fillna('none', inplace=True)
train['alley'].fillna('none', inplace=True)
train['fence'].fillna('none', inplace=True)
train['fireplacequ'].fillna('none', inplace=True)
train['garagecond'].fillna('none', inplace=True) #consider revisiting
train['garagetype'].fillna('none', inplace=True)
train['garagefinish'].fillna('none', inplace=True)
train['garagequal'].fillna('none', inplace=True)
train['garageyrblt'].fillna(0, inplace=True)
train['bsmtexposure'].fillna('none', inplace=True)
train['bsmtfintype1'].fillna('none', inplace=True)
train['bsmtfintype2'].fillna('none', inplace=True)
train['bsmtcond'].fillna('none', inplace=True)
train['bsmtqual'].fillna('none', inplace=True)
train['masvnrtype'].fillna('none', inplace=True)
train['masvnrarea'].fillna(0, inplace=True)

In [8]:
#impute with most common value
train['electrical'].fillna('SBrkr', inplace=True)
print(train['electrical'].value_counts(dropna=False))

SBrkr    1331
FuseA      94
FuseF      27
FuseP       3
Mix         1
Name: electrical, dtype: int64


### Missing value handling: variable lotfrontage
lotfrontage: linear feet of street connected to property

In [9]:
print(train['lotfrontage'].value_counts(
    dropna=False).sort_values(ascending=False).head())

NaN      259
 60.0    143
 70.0     70
 80.0     69
 50.0     57
Name: lotfrontage, dtype: int64


In [10]:
print('street value counts:')
print(train['street'].value_counts(dropna=False))
print('\nlotarea null values: ')

print(train['lotarea'].isnull().sum())
#make sure there aren't unreasonably small lot sizes, could indicate data error
print('\nlotarea smallest lots')
print(train['lotarea'].value_counts(dropna=False).sort_index().head())

street value counts:
Pave    1450
Grvl       6
Name: street, dtype: int64

lotarea null values: 
0

lotarea smallest lots
1300    1
1477    1
1491    1
1526    1
1533    2
Name: lotarea, dtype: int64


In [11]:
temp_df = train.loc[:, (train.columns).isin(['lotfrontage', 'lotarea'])]
temp_df['ratio'] = temp_df['lotfrontage'] / temp_df['lotarea']

print('lotfrontage:lotarea proportion:')
print('ratio mean: %.5f' % temp_df['ratio'].dropna().mean())
print('ratio median: %.5f' % temp_df['ratio'].dropna().median())
print('ratio mode: %.5f' % temp_df['ratio'].dropna().mode())

lotfrontage:lotarea proportion:
ratio mean: 0.00791
ratio median: 0.00771
ratio mode: 0.00833


In [12]:
ratio_mean = (train['lotfrontage'] / train['lotarea']).mean()
train['lotfrontage'].fillna(train['lotarea'] * ratio_mean, inplace=True)
train['lotfrontage'].isnull().sum()

0

In [13]:
#drop id column
train = train.drop(['id'], 1)

In [14]:
#mssubclas to categorical
train = train.replace(
    {'mssubclass':{20:'subclasss20', 30:'subclass30', 40:'subclass40',
                   45:'subclass45', 50:'subclass50', 60:'subclass60',
                   70:'subclass70', 75:'subclass75', 80:'subclass80',
                   85:'subclass85', 90:'subclass90', 120:'subclass120',
                   150:'subclass150', 160:'subclass160', 180:'subclass180',
                   190:'subclass190'},
     'mosold':{1:'_01jan', 2:'_02feb', 3:'_03mar', 4:'_04apr', 5:'_05may',
               6:'_06jun', 7:'_07jul', 8:'_08aug', 9:'_09sep', 10:'_10oct',
               11:'_11nov', 12:'_12dec'}})

In [15]:
#create yr/mo sold feature
train['yrmo_sold'] = train.yrsold.astype(
    str).str.cat(train.mosold.astype(str)) #.astype(int)

In [16]:
#categorical to ordinal
train = train.replace(
    {'street':{'Grvl':1, 'Pave':2},
     'alley':{'none':0, 'Grvl':1, 'Pave':2},
     'utilities':{'ELO':1, 'NoSeWa':2, 'NoSewr':3, 'AllPub':4},
     'exterqual':{'Po':1, 'Fa':2, 'TA':3, 'Gd':4, 'Ex':5},
     'extercond':{'Po':1, 'Fa':2, 'TA':3, 'Gd':4, 'Ex':5},
     'bsmtqual':{'none':0, 'Po':1, 'Fa':2, 'TA':3, 'Gd':4, 'Ex':5},
     'bsmtcond':{'none':0, 'Po':1, 'Fa':2, 'TA':3, 'Gd':4, 'Ex':5},
     'bsmtexposure':{'none':0, 'No':0, 'Mn':2, 'Av':3, 'Gd':4},
     'bsmtfintype1':{'none':0, 'Unf':1, 'LwQ':2, 'Rec':3, 'BLQ':4, 'ALQ':5, 'GLQ':6},
     'bsmtfintype2':{'none':0, 'Unf':1, 'LwQ':2, 'Rec':3, 'BLQ':4, 'ALQ':45, 'GLQ':6},
     'heatingqc':{'Po':1, 'Fa':2, 'TA':3, 'Gd':4, 'Ex':5},
     'kitchenqual':{'Po':1, 'Fa':2, 'TA':3, 'Gd':4, 'Ex':5},
     'functional':{'Sal':1, 'Sev':2, 'Maj2':3, 'Maj1':4, 'Mod':5, 'Min2':6, 'Min1':7, 'Typ':8},
     'fireplacequ':{'none':0, 'Po':1, 'Fa':2, 'TA':3, 'Gd':4, 'Ex':5},
     'garagefinish':{'none':0, 'Unf':1, 'RFn':2, 'Fin':3},
     'garagequal':{'none':0, 'Po':1, 'Fa':2, 'TA':3, 'Gd':4, 'Ex':5},
     'garagecond':{'none':0, 'Po':1, 'Fa':2, 'TA':3, 'Gd':4, 'Ex':5},
     'paveddrive':{'N':1, 'P':2, 'Y':2},
     'poolqc':{'none':0, 'Fa':1, 'TA':2, 'Gd':3, 'Ex':4}})

In [17]:
#possibly subjective
#separate so easy to include or exclude (comment out)
train = train.replace({'lotshape':{'IR3':1, 'IR2':2, 'IR1':3, 'Reg':4}})
train = train.replace({'landcontour':{'Low':1, 'HLS':2, 'Bnk':3, 'Lvl':4}})
#train = train.replace({'lotconfig':{}})
#train = train.replace({'landslope':{'Gtl':1, 'Mod':2, 'Sev':3}})
train = train.replace({'centralair':{'N':0, 'Y':1}})
#train = train.replace({'garagetype':{'none':0, 'CarPort':1, 'Detchd':2},
#                       'BuiltIn':3, 'Basment':4, 'Attchd':5, '2Types':6})

#maybe split this into 2 features
#train = train.replace('fenceqc':{'none':0, 'MnWw':1, 'GdWd':2, 'MnPrv'})

In [18]:
train['saleprice'] = np.log1p(train['saleprice'])

#plt.figure(figsize=(16,8))

#ax1 = plt.subplot(121)
#sns.distplot(train['saleprice'], ax=ax1, fit=norm)
#ax1.set_xlabel('saleprice', size=14)
#ax1.set_title('saleprice distribution', size=18)
#text box
#props = dict(boxstyle='round', color='wheat', alpha=.6)
#textstr1 = 'skew: %f' % (train['saleprice'].skew())
#ax1.text(0.65, 0.95, textstr1, transform=ax1.transAxes, fontsize=14,
#        verticalalignment='top', bbox=props)

#ax2 = plt.subplot(122)
#stats.probplot(train['saleprice'], plot=ax2)
#ax2.set_title('saleprice probability plot')
#ax2.get_lines()[0].set_marker('.')
#ax2.get_lines()[0].set_markerfacecolor(color_b)
#ax2.get_lines()[0].set_markersize(10)
#ax2.get_lines()[1].set_linewidth(0)
#ax2.get_lines()[1].set_marker('.')
#ax2.get_lines()[1].set_markerfacecolor(color_r)
#ax2.get_lines()[1].set_markersize(10)

#plt.show()
#print(stats.normaltest(a=train['saleprice']))

## 2.6 Feature Selection

In [19]:
stop

NameError: name 'stop' is not defined

### PCA

In [24]:
corr_df = train.drop(['saleprice'],1)
corrmat = corr_df.corr()
s = corrmat.unstack()
so = pd.DataFrame(s.sort_values(ascending=False))
so = so.reset_index()
so.columns = ['var1', 'var2', 'corr']
so = so[so['corr'] < 1]
so = so[so['corr'] > 0.5]
so.drop_duplicates(subset='corr', keep='first', inplace=True)
#so.sort_values(by='corr', ascending=False)

In [ ]:
['lotshape', 'landcontour', 'landslope', 'centralair', 'garagetype',
 'street', 'alley', 'utilities', 'exterqual', 'extercond', 'bsmtqual',
 'bsmtcond', 'bsmtexposure', 'bsmtfintype1', 'bsmtfintype2', 'heatingqc',
 'kitchenqual', 'functional', 'fireplacequ', 'garagefinish', 'garagequal',
 'garagecond', 'paveddrive', 'poolqc', 'fence']

In [ ]:
numeric_feats = train.dtypes[train.dtypes != 'object'].index
numeric_feats = numeric_feats.tolist()
numeric_feats.remove('saleprice')

In [ ]:
df_pca = train[numeric_feats]
X = StandardScaler().fit_transform(df_pca)
Xt = X.T
Cx = np.cov(Xt)

eig_val_cov, eig_vec_cov = np.linalg.eig(Cx)

tot = sum(eig_val_cov)
var_exp = [(i / tot)*100 for i in sorted(eig_val_cov, reverse=True)]
cum_var_exp = np.cumsum(var_exp)

plt.plot(sorted(eig_val_cov, reverse=True))
plt.show()
print(cum_var_exp)

In [ ]:
#n_components = 32 #evr 90%

In [ ]:
X_ = X
sklearn_pca = PCA(n_components=32).fit_transform(X_)

In [ ]:
sklearn_pca

In [ ]:
print(np.isnan(sklearn_pca).any())
print(np.isinf(sklearn_pca).any())

In [ ]:
#bstmtfullbath bsmthalfbath fullbath halfbath bedroom kitchen totrmsabvgrd fireplaces garagecars

In [ ]:
df_pca = pd.DataFrame(data=sklearn_pca)

In [ ]:
#df_pca.columns = ['pc1','pc2','pc3','pc4','pc5','pc6','pc7','pc8',
#                  'pc9','pc10']

In [ ]:
df_pca.head()

In [ ]:
print(len(df_pca), len(train['saleprice']))

In [ ]:
price = train['saleprice'].reset_index()

In [ ]:
train_pca = pd.concat([df_pca, price['saleprice']], axis=1)

In [ ]:
print(len(train_pca))

In [ ]:
#train_pca.isnull().value_counts()
train_pca.isnull().sum().sort_values(ascending=False)

In [ ]:
#train_pca = train_pca.drop('index')
train_pca.head()

In [ ]:
categorical_feats = train.columns.tolist()
categorical_feats = [x for x in categorical_feats if x not in numeric_feats]
categorical_feats.remove('saleprice')

In [ ]:
train_cat = train[categorical_feats].reset_index()

In [ ]:
len(train_cat)

In [ ]:
train_cat.head()

In [ ]:
train_cat = train_cat.loc[:, ~(train_cat.columns).isin(['index'])]

In [ ]:
df_final = pd.concat([train_cat, train_pca], axis=1)
df_final.head()
#print(len(df_final.columns))
#pd.concat([df_pca, price['saleprice']], axis=1)

In [ ]:
#create dummies and data/target
df_final = pd.get_dummies(df_final)
X = df_final.loc[:, ~(df_final.columns).isin(['saleprice'])]
#X = train_pca.loc[:, ~(train_pca.columns).isin(['saleprice'])]
y = df_final['saleprice']

#rmse evaluation function
cv = 5

def rmse(model):
    rmse = np.sqrt(-cross_val_score(
        model, X, y, cv=cv, scoring='neg_mean_squared_error'))
    return(rmse)

### Use Lasso/L1 regularization for feature selection

In [ ]:
#lasso = LassoCV(alphas=[0.0001, 0.001, 0.01, 0.1, 1,
#                        0.0005, 0.005, 0.05, 0.5, 5]).fit(X, y)
#alpha = lasso.alpha_
#print('model: lasso')
#print('alpha:', alpha)
#print('rmse score mean:', rmse(lasso).mean())
#print('rmse max - rmse min:', rmse(lasso).max() - rmse(lasso).min())
#print('rmse score array:', rmse(lasso))

In [ ]:
#lasso = LassoCV(alphas=[alpha, alpha*0.25, alpha*0.5, alpha*0.75,
#                        alpha*1.25, alpha*1.5, alpha*1.75, alpha*2]).fit(X, y)
#alpha = lasso.alpha_
#print('model: lasso')
#print('alpha: ', alpha)
#print('rmse score mean:', rmse(lasso).mean())
#print('rmse max - rmse min:', rmse(lasso).max() - rmse(lasso).min())
#print('rmse score array:', rmse(lasso))

In [ ]:
#check residual distribution
#predicted = lasso.predict(X)
#actual = y
#residual = actual - predicted

#plt.figure(figsize=(8,8))
#plt.scatter(predicted, residual)
#plt.axhline(y=0, color=color_r)
#plt.xlabel('predicted')
#plt.ylabel('residual')
#plt.title('error distribution')
#plt.show()

In [ ]:
#coefs = pd.Series(lasso.coef_, index=X.columns)
#n_components = sum(coefs != 0)
#print('lasso n_components: ', n_components)

#features = pd.DataFrame(lasso.coef_, index=X.columns)
#features = features.reset_index()
#features.columns = ['var','coef']
#features = features[features['coef'] != 0]
#features = list(features['var'])

#X = X[features]

# Modeling

### Linear Regression with L2/Ridge Regularization

In [ ]:
#ridge = RidgeCV(
#    alphas=[0.01, 0.05, 0.1, 0.5, 1, 5]
#).fit(X, y)

#alpha = ridge.alpha_
#print('model: ridge')
#print('alpha:', alpha)
#print('rmse score mean:', rmse(ridge).mean())
#print('rmse max - rmse min:', rmse(ridge).max() - rmse(ridge).min())
#print('rmse score array:', rmse(ridge))

In [ ]:
#ridge = RidgeCV(
#    alphas=[alpha, alpha*0.25, alpha*0.5, alpha*0.75,
#            alpha*1.25, alpha*1.5, alpha*1.75, alpha*2]
#).fit(X, y)
#alpha = ridge.alpha_
#print('model: ridge')
#print('alpha:', alpha)
#print('cross_val rmse mean:', rmse(ridge).mean())
#print('rmse max - rmse min:', rmse(ridge).max() - rmse(ridge).min())
#print('rmse score array:', rmse(ridge))

### Linear Regression with L1/Lasso Regularization

In [ ]:
lasso = LassoCV(
    alphas=[0.0001, 0.0005, 0.001, 0.005, 0.01, 0.05, 0.1, 0.5,1, 5]
).fit(X, y)
alpha = lasso.alpha_
print('model: lasso')
print('alpha: ', alpha)
print('rmse score mean:', rmse(lasso).mean())
print('rmse max - rmse min:', rmse(lasso).max() - rmse(lasso).min())
print('rmse score array:', rmse(lasso))

In [ ]:
lasso = LassoCV(alphas=[alpha,
                        alpha*0.25, alpha*0.5, alpha*0.75,
                        alpha*1.25, alpha*1.5, alpha*1.75, alpha*2])

lasso.fit(X, y)
alpha = lasso.alpha_
print('model: lasso')
print('alpha: ', alpha)
print('rmse score mean:', rmse(lasso).mean())
print('rmse max - rmse min:', rmse(lasso).max() - rmse(lasso).min())
print('rmse score array:', rmse(lasso))

### Random Forest Regressor

In [ ]:
rfr = ensemble.RandomForestRegressor()
params = [{'n_estimators':[10, 50, 100, 250, 500],
           'max_features':[1, int(n_components/2), n_components],
           'max_depth':[10,20,30,40]}]

start_time = time.clock()
grid = GridSearchCV(
    estimator=rfr,
    param_grid=params,
    scoring='neg_mean_squared_error'
).fit(X,y)

print('params:', grid.best_params_)
print('runtime: ', time.clock() - start_time)

In [ ]:
n_estimators = grid.best_params_.get('n_estimators')
max_depth = grid.best_params_.get('max_depth')
max_features = grid.best_params_.get('max_features')

start_time = time.clock()
rfr = ensemble.RandomForestRegressor(
    n_estimators=n_estimators,
    max_depth=max_depth,
    max_features=max_features
).fit(X, y)

print('model: random forest')
print('rmse score mean:', rmse(rfr).mean())
print('rmse max - rmse min:', rmse(rfr).max() - rmse(rfr).min())
print('rmse score array:', rmse(rfr))
print('runtime: ', time.clock() - start_time)

### Support Vector Regressor

In [ ]:
#svr = SVR()
#params = [{'C':[0.0001, 0.0005, 0.001, 0.005, 0.01, 0.05, 1, 5, 10],
#           'kernel':['linear']}]

#start_time = time.clock()
#grid = GridSearchCV(
#    estimator=svr,
#    param_grid=params,
#    scoring='neg_mean_squared_error'
#).fit(X, y)

#print('params:', grid.best_params_)
#print('runtime: ', time.clock() - start_time)

In [ ]:
#c = grid.best_params_.get('C')
#start_time = time.clock()
#svr = SVR(C=c, kernel='linear').fit(X, y)

#print('model: support vector')
#print('cross_val rmse mean:', rmse(svr).mean())
#print('rmse max - rmse min:', rmse(svr).max() - rmse(svr).min())
#print('rmse score array:', rmse(svr))
#print('runtime: ', time.clock() - start_time

## XGBoost Regressor

### Vanilla XGB

In [ ]:
start_time = time.clock()
xgbr = XGBRegressor().fit(X, y)
print('model: vanilla xgb_rgr')
print('cross_val rmse mean:', rmse(xgbr).mean())
print('rmse max - rmse min:', rmse(xgbr).max() - rmse(xgbr).min())
print('rmse score array:', rmse(xgbr))
print('runtime: ', time.clock() - start_time)

In [ ]:
start_time = time.clock()
xgbr = XGBRegressor(
    n_estimators=1000,
    learning_rate=0.05
).fit(X, y, verbose=False)

print('model: vanilla xgb_rgr')
print('cross_val rmse mean:', rmse(xgbr).mean())
print('rmse max - rmse min:', rmse(xgbr).max() - rmse(xgbr).min())
print('rmse score array:', rmse(xgbr))
print('runtime: ', time.clock() - start_time)

### Tune Parameters

In [ ]:
xgbr = XGBRegressor()
params = [{'booster':['gbtree', 'gblinear'],
           'max_depth':[2, 3, 5, 10],
           'learning_rate':[0.1, 0.05, 0.01],
           'subsample':[0.25, 0.5, 0.75, 1]}]

start_time = time.clock()
grid = GridSearchCV(
    estimator=xgbr,
    param_grid=params,
    scoring='neg_mean_squared_error'
).fit(X,y)

print('params:', grid.best_params_)
print('runtime: ', time.clock() - start_time)

In [ ]:
booster = grid.best_params_.get('booster')
learning_rate = grid.best_params_.get('learning_rate')
max_depth = grid.best_params_.get('max_depth')
subsample = grid.best_params_.get('subsample')

start_time = time.clock()
xgb_rgr = XGBRegressor(
    n_estimators=1000,
    learning_rate=learning_rate,
    max_depth=max_depth,
    subsample=subsample
).fit(X, y, verbose=False)
#xgb_rgr.fit(X_train, y_train, early_stopping_rounds=5,
#            eval_set=[(X_test, y_test)], verbose=False)

print('model: xgboost regressor')
print('cross_val rmse mean:', rmse(xgbr).mean())
print('rmse max - rmse min:', rmse(xgbr).max() - rmse(xgbr).min())
print('rmse score array:', rmse(xgbr))
print('runtime: ', time.clock() - start_time)